# Software Defined MRR

In [ ]:
import sdmrr
import numpy as np
import scipy.signal as sg
import scipy.optimize as opt
import matplotlib.pyplot as plt
import datetime

from threading import Thread

mrr = sdmrr.SDMRR()

[INFO] [UHD] linux; GNU C++ version 13.3.0; Boost_108600; UHD_4.8.0.HEAD-release
[INFO] [B200] Detected Device: B210
[INFO] [B200] Operating over USB 2.
[INFO] [B200] Initialize CODEC control...
[INFO] [B200] Initialize Radio control...
[INFO] [B200] Performing register loopback test... 
[INFO] [B200] Register loopback test passed
[INFO] [B200] Performing register loopback test... 
[INFO] [B200] Register loopback test passed
[INFO] [B200] Setting master clock rate selection to 'automatic'.
[INFO] [B200] Asking for clock rate 16.000000 MHz... 
[INFO] [B200] Actually got clock rate 16.000000 MHz.


### Calibration

In [2]:
mrr.cal(t90=0.0003, f0=mrr.find_f0(debug=True, freq=22106000.000000))

[INFO] [B200] Asking for clock rate 32.000000 MHz... 
[INFO] [B200] Actually got clock rate 32.000000 MHz.


22105999.999999817


In [ ]:
calfig = plt.figure(figsize=(9,6))

fid = mrr.onepulse(f0, t90, 70, filt=True)
t = np.arange(mrr.NS)/mrr.FS
plt.plot(t, np.real(fid))
plt.plot(t, np.imag(fid))
plt.title("FID")
plt.xlabel("Time (s)")
plt.show()

## CPMG

In [ ]:
def gett2(save = False, amp90 = 0.45, amp180 = 0.9, directory = None):
    
    print("CPMG")
    npulses = 5000
    tr = 500.023e-6
    
    cp = mrr.cpmg_phaseloop(npulses = npulses, amp90 = amp90, cycle_90 = [0], cycle_180 = [1], amp180 = amp180, raw = True) # 
    
    #Get ebr1 amplitudes
    width = 200
    threshold = 1000
    tr_samps = tr*mrr.FS
    t90_samps = int(mrr.caldict['t90'] * mrr.FS)
    mags_abs = np.zeros(npulses)
    mags_r = np.zeros(npulses)

    for i in range(npulses):
        start = int((i+1) * tr_samps - width/2) + t90_samps
        mags_abs[i] = np.average(np.sum(np.abs(cp[:,start:start+width]),axis=0))

    cfig = plt.figure(figsize=(9,5))
    plt.plot(mags_abs[threshold:])
    plt.legend([str(mrr.caldict['f0'])])
    t2 = mrr.get_t2(mags_abs[threshold:], tr=500.2e-6)
    plt.xlabel("T2 = " + str(t2))
    try:
        plt.savefig(directory + "Figures/cpmg" + datetime.datetime.now().isoformat() + ".png")
    except Exception as e:
        print(e)
    plt.close(cfig)
    
    if(save):
        try:
            np.save(directory + datetime.datetime.now().isoformat() + '.npy', cp, allow_pickle=True)
        except Exception as e:
            print(e)
    print("T2: %0.2f" %  (t2))
    return t2

In [ ]:
gett2(save = False, amp90 = 0.45, amp180 = 0.9, directory = None)